# Lab Course Machine Learning
## Exercise Sheet 8(2)
##### January, 2022  
##### Kenechukwu Ejimofor
###### Data Analytics
<center>
<b>
Information Systems and Machine Learning Lab<br>
University of Hildesheim<br>
</b>
<br>
<br>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Universit%C3%A4t_Hildesheim_logo.svg/1200px-Universit%C3%A4t_Hildesheim_logo.svg.png" height="10%" width="10%">
</center>



# Import Libraries

In [1]:
import torch
import cv2
import numpy
import os, glob
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch import optim
from torchsummary import summary
import matplotlib.pyplot as plt
from torch.autograd import Variable



In [2]:

random_seed = 3116
torch.manual_seed(random_seed)

# Setting Up the model

In [3]:
import torch.nn.functional as F


class Convnet(torch.nn.Module):

    def __init__(self):
        super(Convnet, self).__init__()
        self.normalization = nn.BatchNorm2d(3)
        self.convlayer1 = nn.Sequential(
        nn.Conv2d(3, 24, kernel_size=5, stride=2, padding=0),
        nn.ReLU())
            
        self.convlayer2 = nn.Sequential(
        nn.Conv2d(24, 36, kernel_size=5, stride=2, padding=0),
        nn.ReLU())
            
        self.convlayer3 = nn.Sequential(
        nn.Conv2d(36, 48, kernel_size=5, stride=2, padding=0),
        nn.ReLU())

        self.convlayer4 = nn.Sequential(
        nn.Conv2d(48, 64, kernel_size=3, stride=1, padding=0),
        nn.ReLU())
        
        self.convlayer5 = nn.Sequential(
        nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
        nn.ReLU())

        #self.vectorized = nn.Flatten(start_dim=1)
        self.fc1 = nn.Linear(1152, 1164, bias=True)
        self.fc2 = nn.Linear(1164, 100, bias=True)
        self.fc3 = nn.Linear(100, 50, bias=True)
        self.fc4 = nn.Linear(50, 10)
        self.vehicle_control = nn.Linear(10, 1)


    def forward(self, out):
        out = self.normalization(out)
        out = self.convlayer1(out)
        out = self.convlayer2(out)
        out = self.convlayer3(out)
        out = self.convlayer4(out)
        out = self.convlayer5(out)  
        #out = out.view(out.size(0), -1)
        out = out.reshape(out.size(0), -1)        
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = F.relu(self.fc3(out))
        out = F.relu(self.fc4(out))
        out =  self.vehicle_control(out) 
        
        return out

        #return F.softmax(out, dim=1)

model = Convnet()
#model

if torch.cuda.is_available():
    model.cuda()
    
summary(model, (3, 66, 200)) 

Layer (type:depth-idx)                   Output Shape              Param #
├─BatchNorm2d: 1-1                       [-1, 3, 66, 200]          6
├─Sequential: 1-2                        [-1, 24, 31, 98]          --
|    └─Conv2d: 2-1                       [-1, 24, 31, 98]          1,824
|    └─ReLU: 2-2                         [-1, 24, 31, 98]          --
├─Sequential: 1-3                        [-1, 36, 14, 47]          --
|    └─Conv2d: 2-3                       [-1, 36, 14, 47]          21,636
|    └─ReLU: 2-4                         [-1, 36, 14, 47]          --
├─Sequential: 1-4                        [-1, 48, 5, 22]           --
|    └─Conv2d: 2-5                       [-1, 48, 5, 22]           43,248
|    └─ReLU: 2-6                         [-1, 48, 5, 22]           --
├─Sequential: 1-5                        [-1, 64, 3, 20]           --
|    └─Conv2d: 2-7                       [-1, 64, 3, 20]           27,712
|    └─ReLU: 2-8                         [-1, 64, 3, 20]           --
├

Layer (type:depth-idx)                   Output Shape              Param #
├─BatchNorm2d: 1-1                       [-1, 3, 66, 200]          6
├─Sequential: 1-2                        [-1, 24, 31, 98]          --
|    └─Conv2d: 2-1                       [-1, 24, 31, 98]          1,824
|    └─ReLU: 2-2                         [-1, 24, 31, 98]          --
├─Sequential: 1-3                        [-1, 36, 14, 47]          --
|    └─Conv2d: 2-3                       [-1, 36, 14, 47]          21,636
|    └─ReLU: 2-4                         [-1, 36, 14, 47]          --
├─Sequential: 1-4                        [-1, 48, 5, 22]           --
|    └─Conv2d: 2-5                       [-1, 48, 5, 22]           43,248
|    └─ReLU: 2-6                         [-1, 48, 5, 22]           --
├─Sequential: 1-5                        [-1, 64, 3, 20]           --
|    └─Conv2d: 2-7                       [-1, 64, 3, 20]           27,712
|    └─ReLU: 2-8                         [-1, 64, 3, 20]           --
├

# Loading the Dataset

In [4]:
img_path = 'C:/Users/Kenechukwu Ejimofor/Downloads/archive (1)/driving_dataset'

angles_path = 'C:/Users/Kenechukwu Ejimofor/Downloads/archive (1)/driving_dataset/angles.csv'


class Dataset(Dataset):
    def __init__(self):
        
        self.filepath = img_path        
        self.steerangles =  pd.read_csv (self.filepath+'/angles.txt',sep=' ', header = None)
        self.steerangles.columns = ['Image_ID','Steer_angles']
        self.steerangles.to_csv (angles_path, index=None)
        self.steer_angles=pd.DataFrame(self.steerangles, index=None, columns=None)
        
        self.image_ID = self.steer_angles.iloc[:,0]
        self.Steer_Ang = self.steer_angles.iloc[:,1]
        
        # incase data transformations are to be done
        #self.transform = transform
        
    def __len__(self):
        return len(self.image_ID)
    
    def __getitem__(self, index):
                       
        img = cv2.imread(self.filepath + '/' + self.image_ID[index])
        img = cv2.resize(img, (66,200), interpolation = cv2.INTER_AREA)
        
        steer_labels = self.Steer_Ang[index]
        steer_rad_labels = (steer_labels * np.pi)/180
        steer_rad_labels = torch.tensor(steer_rad_labels)
            
        return (torch.from_numpy(img).float(), steer_labels)


# Initializing and Splitting the dataset

In [5]:
        
data = Dataset()
total_len = data.__len__()
train_split = int(0.44708 * total_len)
valid_split = int(0.3327 * total_len)
test_split = total_len - train_split - valid_split
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(data, (train_split, valid_split, test_split))


print(total_len)
print(train_split)
print(valid_split)
print(test_split)

45406
20300
15106
10000


In [6]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=200, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=200, shuffle=True)
#test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=200, shuffle=True)

train_iter = iter(train_loader)
valid_iter = iter(valid_loader)



# Loss, optimizer and training

In [7]:
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

loss_function = torch.nn.MSELoss()


def normalize(X): #minmaxscaler
    min_X = min(X)
    max_X = max(X)
    return (X - min_X)/(max_X - min_X)

model.train()
for epoch in range(5):
        
    train_iter = iter(train_loader)
    valid_iter = iter(valid_loader)

    running_loss = 0.0
    train_counter = 0
    trainloss_history = []

    val_running_loss = 0.0
    valloss_history = []
    val_counter = 0

    for i in range(0, len(train_loader)):
        images, labels = next(train_iter)
        
        images, labels = images, labels.unsqueeze(-1).float()
        images, labels = images, normalize(labels)
        
        images, labels = images, labels
    
        optimizer.zero_grad()
    
        outputs = model(images.permute(0, 3, 1, 2))

        
        # RMSE Loss
        loss = torch.sqrt(loss_function(outputs, labels))

        loss.backward() 
        optimizer.step()
        
        running_loss += loss.item()
        train_counter += 1
        trainloss_history.append(running_loss)
    
        if i % 100 == 0:
            print("Epoch: {}, count: {}, Training loss: {}".format(epoch+1, i+1, running_loss/train_counter))  
           

        
        running_loss = 0.0   
        train_counter = 0        
        
 
    for i in range(0, len(valid_loader)):
        val_images, val_labels = next(valid_iter)
 
        val_images, val_labels = val_images, val_labels.unsqueeze(-1).float()
        val_images, val_labels = val_images, normalize(val_labels)
        
        val_images, val_labels = val_images, val_labels


        with torch.no_grad():

            outputs = model(val_images.permute(0, 3, 1, 2))

            loss = torch.sqrt(loss_function(outputs, val_labels))

            val_running_loss += loss.item()
            val_counter += 1  
            valloss_history.append(val_running_loss)

        if i % 100 == 0:
            print("Epoch: {}, count: {}, validation loss: {}".format(epoch+1, i+1, val_running_loss/val_counter))  
     
    
        val_running_loss = 0.0   
        val_counter = 0     

print('Finished Training and Validation')


Epoch: 1, count: 1, Training loss: 0.37686508893966675
Epoch: 1, count: 101, Training loss: 0.16716788709163666
Epoch: 1, count: 1, validation loss: 0.07973787933588028
Epoch: 2, count: 1, Training loss: 0.11418464779853821
Epoch: 2, count: 101, Training loss: 0.10310390591621399
Epoch: 2, count: 1, validation loss: 0.33670151233673096
Epoch: 3, count: 1, Training loss: 0.12477435171604156
Epoch: 3, count: 101, Training loss: 0.13601472973823547
Epoch: 3, count: 1, validation loss: 0.25793761014938354
Epoch: 4, count: 1, Training loss: 0.11055180430412292
Epoch: 4, count: 101, Training loss: 0.13409778475761414
Epoch: 4, count: 1, validation loss: 0.09219714254140854
Epoch: 5, count: 1, Training loss: 0.13334177434444427
Epoch: 5, count: 101, Training loss: 0.48123350739479065
Epoch: 5, count: 1, validation loss: 0.17903611063957214
Finished Training and Validation


# RMSE for the test set of images

In [8]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=200, shuffle=True)
test_iter = iter(test_loader)


for epoch in range(1):
    total = 0.0
    correct = 0.0
    count = 0
    test_running_loss = 0.0
    test_loss_list = []
    test_counter = 0

    for i in range(0, len(test_loader)):
        test_images, test_labels = next(test_iter)
        test_images, test_labels = test_images, test_labels.unsqueeze(-1).float()
        #test_images, test_labels = test_images.cuda(), normalize(test_labels).cuda()
        test_images, test_labels = test_images, normalize(test_labels)
        test_images, test_labels = test_images, test_labels
        model.eval()
        with torch.no_grad():
            if torch.cuda.is_available():      
                test_images = test_images.cuda()
            output = model(test_images.permute(0, 3, 1, 2))#.cuda()
            out = torch.argmax(output,1)[0]
            
            total += test_labels.size(0)*test_labels.size(1)
            correct += (out.cpu()==test_labels.cpu()).sum()
            loss = torch.sqrt(loss_function(output, test_labels))

            test_running_loss += loss.item()
            test_counter += 1  
            acc = correct.double()/total * 100
            test_loss_list.append(test_running_loss/test_counter)
        count += 1
            
        if i % 10 == 0:
            print("epoch: {} , i: {}, test loss:{},  accuracy: {}".format(epoch+1, i+1,test_running_loss/test_counter, acc))
          
        count = 0
        

epoch: 1 , i: 1, test loss:0.31584852933883667,  accuracy: 0.5
epoch: 1 , i: 11, test loss:0.22201808812943372,  accuracy: 0.5
epoch: 1 , i: 21, test loss:0.23348773980424517,  accuracy: 0.5
epoch: 1 , i: 31, test loss:0.24758656563297396,  accuracy: 0.5161290322580645
epoch: 1 , i: 41, test loss:0.23457515930257192,  accuracy: 0.5121951219512195


In [9]:
print(f'RMSE for test set: {np.mean(test_loss_list)} ')

RMSE for test set: 0.23607634426537072 


# Hyperparameter Tuning, Regularization with ImageTransformations

todo: 
1. regularization scheme titled, "MixUp"
2. regularization scheme named "Cutout" 
3. batch_size, number_of_layers, kernel_sizes, learning_rate, l1_regularization, l2_regularization coefficients and Random Search or Hyperband.

### Mixup implementation

In [10]:
model.train()
for epoch in range(2):
        
    train_iter = iter(train_loader)
    valid_iter = iter(valid_loader)

    running_loss = 0.0
    train_counter = 0
    trainloss_history = []
    val_running_loss = 0.0
    valloss_history = []
    val_counter = 0
    alpha = 1
    lam = numpy.random.beta(alpha, alpha)
    mimages = Variable(lam * images+ (1. - lam) * images)
    mlabels = Variable(lam * labels + (1. - lam) *labels)
    for i in range(0, len(train_loader)):
        images, labels = next(train_iter)
        
        images, labels = images, labels.unsqueeze(-1).float()
        images, labels = images, normalize(labels)
   


        if torch.cuda.is_available():
            images, labels = images.cuda(), labels#.cuda()
            lam = numpy.random.beta(alpha, alpha)
            mimages = Variable(lam * images+ (1. - lam) * images)
            mlabels = Variable(lam * labels + (1. - lam) *labels)
            
        
        optimizer.zero_grad()
    
        outputs = model(mimages.permute(0, 3, 1, 2))#.cuda()

        loss = loss_function(outputs, mlabels)

        loss.backward() 
        optimizer.step()
        
        running_loss += loss.item()
        train_counter += 1
        trainloss_history.append(running_loss)
    
        if i % 100 == 0:
            print("Epoch: {}, count: {}, Training loss: {}".format(epoch+1, i+1, running_loss/train_counter))  
           

        
        running_loss = 0.0   
        train_counter = 0 
    for i in range(0, len(valid_loader)):
        val_images, val_labels = next(valid_iter)
 
        val_images, val_labels = val_images, val_labels.unsqueeze(-1).float()
        val_images, val_labels = val_images, normalize(val_labels)
        
        val_images, val_labels = val_images, val_labels


        with torch.no_grad():

            outputs = model(val_images.permute(0, 3, 1, 2))

            loss = torch.sqrt(loss_function(outputs, val_labels))

            val_running_loss += loss.item()
            val_counter += 1  
            valloss_history.append(val_running_loss)

        if i % 100 == 0:
            print("Epoch: {}, count: {}, validation loss: {}".format(epoch+1, i+1, val_running_loss/val_counter))  
     
    
        val_running_loss = 0.0   
        val_counter = 0     

print('Finished Training and Validation')

Epoch: 1, count: 1, Training loss: 0.1048712357878685
Epoch: 1, count: 101, Training loss: 0.012987370602786541
Epoch: 1, count: 1, validation loss: 0.15873102843761444
Epoch: 2, count: 1, Training loss: 0.15616750717163086
Epoch: 2, count: 101, Training loss: 0.017174484208226204
Epoch: 2, count: 1, validation loss: 0.1791597157716751
Finished Training and Validation


### Cutout Implementation

In [11]:
def cutout(img):
    #In the original paper the number of holes and length are choosable
    #For the purpose of simplicity we are cutting out just 1 hole and setting a fixed length of pixels
    x1 = np.random.randint(0, int(img.shape[0]*0.8))
    y1 = np.random.randint(0, int(img.shape[1]*0.8))
    width = np.random.randint(1, int(img.shape[0]*0.2)) #pixel width
    length = np.random.randint(1, int(img.shape[1]*0.2)) #pixel length
    img[x1:(x1+width), y1:(y1+length), :] = np.zeros((width, length, img.shape[2])) #cutout by setting square equal to zero
    return img

class Dataset2(Dataset):
    def __init__(self):
        
        self.filepath = img_path        
        self.steerangles =  pd.read_csv (self.filepath+'/angles.txt',sep=' ', header = None)
        self.steerangles.columns = ['Image_ID','Steer_angles']
        self.steerangles.to_csv (angles_path, index=None)
        self.steer_angles=pd.DataFrame(self.steerangles, index=None, columns=None)
        
        self.image_ID = self.steer_angles.iloc[:,0]
        self.Steer_Ang = self.steer_angles.iloc[:,1]
        
        # incase data transformations are to be done
        #self.transform = transform   
    def __len__(self):
        return len(self.image_ID)
    
    def __getitem__(self, index):
                       
        img = cv2.imread(self.filepath + '/' + self.image_ID[index])
        img = cv2.resize(img, (66,200), interpolation = cv2.INTER_AREA)
        img = cutout(img)
        steer_labels = self.Steer_Ang[index]
        steer_rad_labels = (steer_labels * np.pi)/180
        steer_rad_labels = torch.tensor(steer_rad_labels)
            
        return (torch.from_numpy(img).float(), steer_labels)

In [12]:
        
data2 = Dataset2()
total_len = data2.__len__()
train_split = int(0.44708 * total_len)
valid_split = int(0.3327 * total_len)
test_split = total_len - train_split - valid_split
train_dataset1, valid_dataset1, test_dataset1 = torch.utils.data.random_split(data2, (train_split, valid_split, test_split))


print(total_len)
print(train_split)
print(valid_split)
print(test_split)

45406
20300
15106
10000


In [13]:
train_loader1 = torch.utils.data.DataLoader(train_dataset1, batch_size=200, shuffle=True)
valid_loader1 = torch.utils.data.DataLoader(valid_dataset1, batch_size=200, shuffle=True)
#test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=200, shuffle=True)

train_iter1 = iter(train_loader1)
valid_iter1 = iter(valid_loader1)



In [14]:
model.train()
for epoch in range(2):
        
    train_iter = iter(train_loader1)
    valid_iter = iter(valid_loader1)

    running_loss = 0.0
    train_counter = 0
    trainloss_history = []

    val_running_loss = 0.0
    valloss_history = []
    val_counter = 0

    for i in range(0, len(train_loader1)):
        images, labels = next(train_iter)
        
        images, labels = images, labels.unsqueeze(-1).float()
        images, labels = images, normalize(labels)
        
        images, labels = images, labels
    
        optimizer.zero_grad()
    
        outputs = model(images.permute(0, 3, 1, 2))

        
        # RMSE Loss
        loss = torch.sqrt(loss_function(outputs, labels))

        loss.backward() 
        optimizer.step()
        
        running_loss += loss.item()
        train_counter += 1
        trainloss_history.append(running_loss)
    
        if i % 100 == 0:
            print("Epoch: {}, count: {}, Training loss: {}".format(epoch+1, i+1, running_loss/train_counter))  
           

        
        running_loss = 0.0   
        train_counter = 0        
        
 
    for i in range(0, len(valid_loader1)):
        val_images, val_labels = next(valid_iter)
 
        val_images, val_labels = val_images, val_labels.unsqueeze(-1).float()
        val_images, val_labels = val_images, normalize(val_labels)
        
        val_images, val_labels = val_images, val_labels


        with torch.no_grad():

            outputs = model(val_images.permute(0, 3, 1, 2))

            loss = torch.sqrt(loss_function(outputs, val_labels))

            val_running_loss += loss.item()
            val_counter += 1  
            valloss_history.append(val_running_loss)

        if i % 100 == 0:
            print("Epoch: {}, count: {}, validation loss: {}".format(epoch+1, i+1, val_running_loss/val_counter))  
     
    
        val_running_loss = 0.0   
        val_counter = 0     

print('Finished Training and Validation')

Epoch: 1, count: 1, Training loss: 0.42045167088508606
Epoch: 1, count: 101, Training loss: 0.12994158267974854
Epoch: 1, count: 1, validation loss: 0.2545176148414612
Epoch: 2, count: 1, Training loss: 0.07964934408664703
Epoch: 2, count: 101, Training loss: 0.6221649646759033
Epoch: 2, count: 1, validation loss: 0.22404688596725464
Finished Training and Validation


### Hyperparameter tuning

Kindly find in the next notebook the implementation